In [35]:
# import the libraries
import requests
import csv
from bs4 import BeautifulSoup
import pandas as pd
import plotly.plotly as py
import plotly.graph_objs as go
import numpy as np
from sklearn import manifold
from sklearn.metrics import euclidean_distances
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt 

In [36]:
# convert model names to brands

# open the models file and use that to replace model names with brand names. 
# format of file is brand, model
#file = open('models.csv', 'r')
file = open('models.csv', 'r', encoding='mac_roman', newline='')

csv_file = csv.reader(file)

# store the brand names
brands = []
for i in csv_file:
    brands.append(i)
#     parsedData = parsedData.replace(i[1], i[0])

print('Finished storing this csv file into a list')
# close the file
file.close()

Finished storing this csv file into a list


In [38]:
with open('finalcomments.csv') as f:
    data=[tuple(line) for line in csv.reader(f)]
print(data[-1])

('October  6, 2018 12:55AM', 'henryn', " a few years back, before i returned to work, i played around quite a bit with dvd movies.  as long as it's not copy protected, it is fairly easy to copy to your hard drive, and then change the region coding.  if it's copy protected, that is a different story. and even if you do remove the region coding, you still have the problem of ntsc vs pal.  your dvd player and tv may handle pal -- and then again, maybe not.")


In [37]:
unique_brands = []
for b in brands:
    if b[0] not in unique_brands:
        unique_brands.append(b[0])
unique_brands.remove('car')
unique_brands.remove('hyndai kia')
unique_brands.remove('hyundai,')
unique_brands.remove('hyundai.')
unique_brands.remove('kia,')
unique_brands.remove('kia.')
unique_brands.remove('mercedes benz')
unique_brands.remove('mercedes-benz')
unique_brands.remove('nissan.')
unique_brands.remove('problem')
unique_brands.remove('seat')
unique_brands.remove('toyata')
unique_brands.remove('volkwagen')

### Get comments

In [39]:
comments = []
for c in data:
    comments.append(c[2])
comments.pop(0)

'comment'

In [40]:
tot_number_of_comments = len(comments)

In [41]:
tot_number_of_comments

77079

## Details of analysis
Our idea for how to define what makes a car "aspirational" is to compute the lift between the brand and a set of aspirational words. Practicly, this is done by getting the number of comments for each brand name where that brand name and any aspirational word is mentioned. These numbers are then used to calculate lift.

After doing this calculation, we found that the lift between brand and aspirational words were very similar for every brand, which does not give us much insight to which cars are aspirational. So then we multiplied the lift for each brand by the number of comments that mentioned that brand. This of course biased our definition of top aspired brands to those of which were mentioned the most.

### Get dictionary of brand name and count

In [42]:
unique_brands_dict = {}
for b in unique_brands:
    for c in comments:
        if b in c:
            if b in unique_brands_dict:
                unique_brands_dict[b] += 1
            else:
                unique_brands_dict[b] = 1

Get top ten brand names

### Get dictionary of brand name and count of comments with aspirational words

In [43]:
brands_aspiration_dict = {}
for b in unique_brands:
    for c in comments:
        if b in c and\
        ('want' in c\
        or 'choose' in c\
        or 'prefer' in c\
        or 'require' in c\
        or 'needs' in c\
        or 'passion' in c\
        or 'incline' in c\
        or 'tend' in c\
        or 'itch' in c\
        or 'fancy' in c\
        or 'wish' in c\
        or 'great' in c\
        or 'looking' in c\
        or 'like' in c\
        or 'good' in c\
        or 'best' in c\
        or 'nice' in c\
        or 'worth' in c\
        or 'love' in c):
            if b in brands_aspiration_dict:
                brands_aspiration_dict[b] += 1
            else:
                brands_aspiration_dict[b] = 1

In [44]:
brands_aspiration_dict
brands_aspiration = sorted(brands_aspiration_dict.items(), key=operator.itemgetter(1))
brands_aspiration.reverse()
brands_aspiration[:10]

[('ford', 2698),
 ('dodge', 1818),
 ('chevrolet', 1724),
 ('bmw', 1633),
 ('honda', 1554),
 ('audi', 1495),
 ('volkswagen', 1398),
 ('mercedes', 1214),
 ('cadillac', 1104),
 ('toyota', 1103)]

### Total count of comments with aspirirational words

In [47]:
asp_count = 0
for c in comments:
    if ('want' in c\
    or 'choose' in c\
    or 'prefer' in c\
    or 'require' in c\
    or 'needs' in c\
    or 'passion' in c\
    or 'incline' in c\
    or 'tend' in c\
    or 'itch' in c\
    or 'fancy' in c\
    or 'wish' in c\
    or 'great' in c\
    or 'looking' in c\
    or 'like' in c\
    or 'good' in c\
    or 'best' in c\
    or 'nice' in c\
    or 'worth' in c\
    or 'love' in c):
        asp_count += 1

### Calculate lift for each brand and asp word

In [48]:
# tot_number_of_comments
# unique_brands_dict
# brands_aspiration_dict
# asp_count

brand_asp_lifts_dict = {}
for b in brands_aspiration_dict:
    
    brand_asp_count = brands_aspiration_dict[b]
    brand_count = unique_brands_dict[b]
    
    lift = (tot_number_of_comments * brand_asp_count) / (brand_count * asp_count)
    brand_asp_lifts_dict[b] = lift

In [56]:
brand_asp_lifts = sorted(brand_asp_lifts_dict.items(), key=operator.itemgetter(1))
brand_asp_lifts.reverse()
brand_asp_lifts[:27]

[('saturn', 1.5569551711115317),
 ('infiniti', 1.5275401299958844),
 ('sedan', 1.5047919455775787),
 ('kia', 1.4837940535409657),
 ('acura', 1.4829089346461937),
 ('audi', 1.4591991779167053),
 ('nissan', 1.4391863616287162),
 ('cadillac', 1.419295836242414),
 ('mercury', 1.4100796295135671),
 ('mercedes', 1.4077002820247473),
 ('mazda', 1.4000464380159718),
 ('lincoln', 1.3941465263552217),
 ('bmw', 1.3889266677215755),
 ('toyota', 1.3878398115029988),
 ('hyundai', 1.3699064206955167),
 ('subaru', 1.3661448035201458),
 ('chrysler', 1.3526003712836996),
 ('chevrolet', 1.352475041246998),
 ('volkswagen', 1.3494553066145467),
 ('dodge', 1.349352685951398),
 ('volvo', 1.3465528833598301),
 ('honda', 1.3462213629847801),
 ('mitsubishi', 1.324676113826253),
 ('buick', 1.3203393765488458),
 ('ford', 1.3182953184506796),
 ('pontiac', 1.3050705294026477),
 ('suzuki', 1.14895377219624)]

### Multiply lift by count for each brand

In [50]:
aspirations = {}
for b in unique_brands:
    aspirations[b] = brand_asp_lifts_dict[b] * unique_brands_dict[b]

In [57]:
top_aspirations = sorted(aspirations.items(), key=operator.itemgetter(1))
top_aspirations.reverse()
top_aspirations

[('ford', 5424.785235424547),
 ('dodge', 3655.3964262423374),
 ('chevrolet', 3466.3935307160555),
 ('bmw', 3283.4226424938042),
 ('honda', 3124.579783487675),
 ('audi', 3005.9503065084127),
 ('volkswagen', 2810.915403678101),
 ('mercedes', 2440.9522890309117),
 ('cadillac', 2219.7786878831353),
 ('toyota', 2217.7680187817923),
 ('nissan', 2070.9891743837225),
 ('buick', 1584.407251858615),
 ('chrysler', 1566.3112299465242),
 ('sedan', 1178.252093387244),
 ('kia', 1047.5586017999217),
 ('hyundai', 997.2918742663362),
 ('acura', 920.8864484152863),
 ('pontiac', 914.854441111256),
 ('lincoln', 904.8010956045389),
 ('subaru', 890.726411895135),
 ('mazda', 834.4276770575192),
 ('saturn', 655.4781270379549),
 ('volvo', 591.1367157949654),
 ('infiniti', 432.2938567888353),
 ('mitsubishi', 225.19493935046302),
 ('mercury', 217.15226294508935),
 ('suzuki', 32.17070562149472)]

### Another analysis method
Because our previous analysis ended up so biased towards brands that had been commented the most, we wanted to try a different method and compare results.<br>
<br>
For each brand, we took the number of comments that had the brand name and an aspirational word, and divided it by the number of comments that brand had in it. This 

In [59]:
another_asp_dict = {}
for b in unique_brands_dict:
    another_asp_dict[b] = brands_aspiration_dict[b]/unique_brands_dict[b]

In [60]:
another_asp = sorted(another_asp_dict.items(), key=operator.itemgetter(1))
another_asp.reverse()
another_asp[:27]

[('saturn', 0.7743467933491687),
 ('infiniti', 0.7597173144876325),
 ('sedan', 0.7484035759897829),
 ('kia', 0.7379603399433428),
 ('acura', 0.7375201288244766),
 ('audi', 0.7257281553398058),
 ('nissan', 0.7157748436414176),
 ('cadillac', 0.7058823529411765),
 ('mercury', 0.7012987012987013),
 ('mercedes', 0.7001153402537486),
 ('mazda', 0.6963087248322147),
 ('lincoln', 0.6933744221879815),
 ('bmw', 0.6907783417935702),
 ('toyota', 0.6902377972465582),
 ('hyundai', 0.6813186813186813),
 ('subaru', 0.6794478527607362),
 ('chrysler', 0.6727115716753023),
 ('chevrolet', 0.6726492391728444),
 ('volkswagen', 0.671147383581373),
 ('dodge', 0.6710963455149501),
 ('volvo', 0.6697038724373576),
 ('honda', 0.6695389918138733),
 ('mitsubishi', 0.6588235294117647),
 ('buick', 0.6566666666666666),
 ('ford', 0.6556500607533414),
 ('pontiac', 0.6490727532097005),
 ('suzuki', 0.5714285714285714)]